In [ ]:
# !uv pip install dash

In [ ]:
"""
Step 1. Load data
"""
from utils.trial import load_trial

# Edit these values
SUBJECTS_DIR = "subjects"
SUBJECT_NAME = "ncbc_s15"
TRIAL_NAME = "TM_p9"
RESULTS_FILE = "results.json"
# ================================

trial = load_trial(SUBJECTS_DIR, SUBJECT_NAME, TRIAL_NAME)
SUBJECT_WEIGHT = trial.subject_weight

sensors = trial.load_sensors()
treadmill_unaligned = trial.load_treadmill()

In [ ]:
from typing import List, Tuple
import threading
import webbrowser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dash
from dash import *
from dash import dcc, html, ctx
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import datetime

from utils.sensor import SensorData

In [ ]:
def append_elapsed_time(df: pd.DataFrame) -> pd.DataFrame: 
    dt = []
    start_time = datetime.datetime.fromtimestamp(df.time.values[0])
    for x in df.time:
        xt = datetime.datetime.fromtimestamp(x)
        diff = xt - start_time
        dt.append(diff.total_seconds())
    df['elapsed_time'] = dt

    return df

sensors.left_df = append_elapsed_time(sensors.left_df)
sensors.right_df = append_elapsed_time(sensors.right_df)    
sensors.waist_df = append_elapsed_time(sensors.waist_df)    


In [ ]:
# add 

In [ ]:
app = dash.Dash(__name__)

a = 2000
b = 5000
left_df = sensors.left_df.iloc[a:b].copy()
right_df = sensors.right_df.iloc[a:b].copy()
waist_df = sensors.waist_df.iloc[a:b].copy()

window_size = 3
fps = 100
window_frame = round(window_size * fps)  # Convert window size to frames


acc_fig = go.Figure(
    data=[go.Scatter(x=np.arange(0, window_size, 1/fps), 
                    y=left_df['accel_filtered'][0:window_frame], mode='lines', name='Left Ankle'), 
        go.Scatter(x=np.arange(0, window_size, 1/fps), 
                    y=right_df['accel_filtered'][0:window_frame], mode='lines', name='Right Ankle'), 
        go.Scatter(x=np.arange(0, window_size, 1/fps), 
                    y=waist_df['accel_filtered'][0:window_frame], mode='lines', name='Waist')],

    layout=go.Layout(
        xaxis=dict(range=[0, window_size]),
        # yaxis=dict(range=[0, waist_df['accel_filtered'].max() * 1.1]),
        xaxis_title='Time (s)',
        yaxis_title='Acceleration (m/s²)',
        font=dict(family="Arial, sans-serif"),
        title_text='Sensor Acceleration Inputs',
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                        method="animate",
                        args=[None, {"frame": {"duration": 0.1/fps, "redraw": True},
                                    "fromcurrent": True, "transition": {"duration": 0}}]), 
                                    
                    dict(label="Pause",
                        method="animate",
                        args=[[None], {"frame": {"duration": 0, "redraw": False},
                                        "mode": "immediate", "transition": {"duration": 0}}]) 
            ])
        ],),

    frames=[go.Frame(data=[
                    go.Scatter(x=np.arange(0, window_size, 1/fps),
                        y=left_df['accel_filtered'][k:k+window_frame], 
                        mode='lines', name='Left Ankle'), 
                    go.Scatter(x=np.arange(0, window_size, 1/fps),
                        y=right_df['accel_filtered'][k:k+window_frame], 
                        mode='lines', name='Right Ankle'), 
                    go.Scatter(x=np.arange(0, window_size, 1/fps),
                        y=waist_df['accel_filtered'][k:k+window_frame], 
                        mode='lines', name='Waist')
                ],)
        for k in range(0, len(waist_df)-window_frame)]
)


parsed_steps_fig = go.Figure(
    layout=go.Layout(
        xaxis=dict(range=[0, window_size]),
        # yaxis=dict(range=[0, waist_df['accel_filtered'].max() * 1.1]),
        # xaxis_title='Time (s)',
        # yaxis_title='Acceleration (m/s²)',
        # font=dict(family="Arial, sans-serif"),
        title_text='Model Outputs',
        # title_text_x=0.5,
)
)

app.layout = html.Div([
    html.H1(children='Vetta Realtime vGRF Estimation',
                style={'textAlign': 'center', 
                    # 'marginBottom': 20, 
                    'fontFamily': 'Arial'}),

        dcc.Tab(label='Acc Inputs',
                children=[
            dcc.Graph(id='Acc-Inputs', 
                        figure=acc_fig,), 
            dcc.Graph(id='Model-Outputs', 
                        figure=parsed_steps_fig,), 
        ]),
])


# @app.callback(
#     Output('Model-Outputs', 'figure'),
#     Output('Acc-Inputs', 'figure'),
#     # Input('window-slider', 'value'),
#     Input('Acc-Inputs', 'relayoutData'),

#     prevent_initial_call=True
#     )

# def fig_playback(relayoutData):  # 
#     ''' Update the input figure based on the selected window size. '''

#     parsed_steps_fig.add_trace(
#         go.Scatter(
#             x=acc_fig.data[2].x, 
#             y=acc_fig.data[0].y,
#             mode='lines',
#             name='Waist',
#             line=dict(color='blue')
#         )
#     )

#     # acc_fig.update_layout(updatemenus=[dict(
#     #                     type="buttons",

#     #                     buttons=[dict(label="Play",
#     #                     args=['marker.color', 'red']), 
                                    
#                     # dict(label="Pause",
#                     #     method="animate",
#                     #     args=[[None], {"frame": {"duration": 0, "redraw": False},
#                     #                     "mode": "immediate", "transition": {"duration": 0}}]) 
#         #     ])
#         # ],),

#     # print(acc_fig.data[2].y)
#     print(f"Relayout Data: {relayoutData}")

#     return parsed_steps_fig, acc_fig

# def open_browser():
#     webbrowser.open_new("http://127.0.0.1:8050/")

# threading.Timer(1, open_browser).start()
app.run(debug=True, use_reloader=False, host='127.0.0.1', port=8050)

In [ ]:
raise StopIteration

In [ ]:
np.random.uniform(1, 5, 50)

In [ ]:
import plotly.graph_objects as go
import numpy as np
import dash
from dash import dcc, html, ctx
from dash.dependencies import Input, Output, State

# construct a figure with frames
frames=[go.Frame(name=n, data=go.Scatter(y=np.sin(np.linspace(n, n+3, 100))))
        for n in range(100)]
fig = go.Figure(data=frames[0].data, frames=frames)
# fig = fig.update_layout(
#     updatemenus=[{"buttons": [{"args": [None, {"frame": {"duration": 500, "redraw": True}}],
#                                "label": "&#9654;",
#                                "method": "animate",},],
#                   "type": "buttons",}],
#     sliders=[{"steps": [{"args": [[f.name],{"frame": {"duration": 0, "redraw": True}, "mode": "immediate",},],
#                          "label": f.name, "method": "animate",}
#                         for f in frames],
#              }],)


# Build App
app = dash.Dash(__name__)
app.layout = html.Div(
    [
     html.Button("Play", id="dashPlay", n_clicks=0),
     html.Div(id="whichframe", children=[]),
     dcc.Graph(id="graph", figure=fig), 
     dcc.Slider(id="dashSlider", min=0, max=len(frames)-1, value=0, marks={i:{"label":str(i)} for i in range(len(frames))}),
     dcc.Interval(id="animateInterval", interval=400, n_intervals=0, disabled=True),
     
    ],
)

# core update of figure on change of dash slider    
@app.callback(
    Output("whichframe", "children"),
    Output("graph", "figure"),
    Input("dashSlider", "value"),
)
def setFrame(frame):
    if frame:
        tfig = go.Figure(fig.frames[frame].data, frames=fig.frames, layout=fig.layout)
        try:
            tfig.layout['sliders'][0]['active'] = frame
        except IndexError:
            pass
        return frame, tfig
    else:
        return 0, fig

# start / stop Interval to move through frames
@app.callback(
    Output("animateInterval","disabled"),
    Input("dashPlay", "n_clicks"),
    State("animateInterval","disabled"),
)
def play(n_clicks, disabled):
    return not disabled
    
@app.callback(
    Output("dashSlider", "value"),
    Input("animateInterval", "n_intervals"),
    State("dashSlider", "value")
)
def doAnimate(i, frame):
    if frame < (len(frames)-1): 
        frame += 1
    else:
        frame = 0
    return frame

# Run app and display result inline in the notebook
app.run(mode="inline")

In [ ]:
import plotly.graph_objects as go

a = 2000
b = 5000
left_df = sensors.left_df.iloc[a:b].copy()
right_df = sensors.right_df.iloc[a:b].copy()

window_size = 5
fps = 100
window_frame = round(window_size * fps)  # Convert window size to frames

# Initialize the figure with the first frame's data
fig = go.Figure(
    data=[go.Scatter(x=np.arange(0, window_size, 1/fps), 
                     y=left_df['accel_filtered'][0:window_frame], mode='lines', name='Left Ankle'), 
          go.Scatter(x=np.arange(0, window_size, 1/fps), 
                     y=right_df['accel_filtered'][0:window_frame], mode='lines', name='Right Ankle')],
    layout=go.Layout(
        xaxis=dict(range=[-window_size, window_size]),
        yaxis=dict(range=[0, df['accel_filtered'].max() * 1.1]),
        title_text='Graph Objects Animation',
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 1/fps, "redraw": True},
                                       "fromcurrent": True, "transition": {"duration": 0}}]), 
                    dict(label="Pause",
                          method="animate",
                          args=[[None], {"frame": {"duration": 0, "redraw": False},
                                         "mode": "immediate", "transition": {"duration": 0}}]) 
            ])
        ]
    ),
    
    frames=[go.Frame(data=[go.Scatter(x=np.arange(-window_size, window_size, 1/fps),
                                      y=left_df['accel_filtered'][k-window_frame:k+window_frame], 
                                      mode='lines', name='Left Ankle'), 
                           go.Scatter(x=np.arange(-window_size, window_size, 1/fps),
                                      y=right_df['accel_filtered'][k-window_frame:k+window_frame], 
                                      mode='lines', name='Right Ankle')],)
            for k in range(window_frame, len(df)-window_frame)]
)

fig.show()

In [ ]:
import dash
import plotly.graph_objects as go
from dash import Input, Output, Patch, callback, html, dcc


app = dash.Dash(__name__)
fig = go.Figure(
    go.Scatter(x=[0, 1, 2], y=[4, 5, 6]),
)

app.layout = html.Div([
    html.Button(
        "Update Sublot",
        id="update-trace"
    ),
    dcc.Graph(
        figure=fig,
        id="my-fig"
    ),
])


@callback(
    Output("my-fig", "figure"),
    Input("update-trace", "n_clicks"),
    prevent_initial_call=True
)
def my_callback(clicks):

    # Creating a Patch object
    patched_figure = Patch()
    x = clicks * 0.1
    patched_figure['layout']['shapes'] = [{'type': 'line', 'x0': x, 'x1': x, 'xref': 'x', 'y0': 0, 'y1': 1, 'yref': 'y domain'}]
    return patched_figure


if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
import plotly.graph_objects as go
import numpy as np
# Generate curve data
t = np.linspace(-1, 1, 100)
x = t + t ** 2
y = t - t ** 2
xm = np.min(x) - 1.5
xM = np.max(x) + 1.5
ym = np.min(y) - 1.5
yM = np.max(y) + 1.5
N = 25
s = np.linspace(-1, 1, N)
xx = s + s ** 2
yy = s - s ** 2


# Create figure
fig = go.Figure(
    data=[go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue")),
          go.Scatter(x=[xx[0]], y=[yy[0]],
                     mode="markers",
                     marker=dict(color="red", size=10))])
fig.update_layout(width=600, height=450,
        xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
        yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
        title_text="Kinematic Generation of a Planar Curve", title_x=0.5,
        updatemenus = [dict(type = "buttons",
        buttons = [
            dict(
                args = [None, {"frame": {"duration": 10, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 10}}],
                label = "Play",
                method = "animate",

                )])])

fig.update(frames=[go.Frame(
                        data=[go.Scatter(
                                   x=[xx[k]],
                                   y=[yy[k]])],
                        traces=[1]) # fig.data[1] is updated by each frame
        for k in range(N)])



fig.show()

In [ ]:
# from dash import *
# import pandas as pd
# import plotly.express as px
# # import dash_bootstrap_components as dbc

# import json
# data = {

#     "x": [1,2,3,4,5],
#     "y": [1,2,3,4,5],
#     "time": [1,2,3,4,5]

# }

# df = pd.DataFrame(data)

# app = Dash() #external_stylesheets=[dbc.themes.SLATE])
# app.layout = html.Div([
#     html.Div(
#         className="app-header",
#         children=[
#             html.H1('Overview of the DashBoard', className="app-header--title"),
#             html.Button(id='syncData', style={'display': 'none'}),
#             dcc.Graph(id ="myfig",figure = px.scatter(df, x='x', y='y',  animation_frame="time", range_x=[0,5], range_y=[0,5])),
#             dcc.Store(id='currentFrame', storage_type='local', data=0),
#             html.Div(id='toShow')
#         ]
#     )])

# app.clientside_callback(
#     """
#         async function (graph, target) {
#             var myGraph = document.querySelector(`#${graph} .js-plotly-plot`)
#             const delay = ms => new Promise(res => setTimeout(res, ms));
#             while (!myGraph) {
#                 await delay(100)
#                 myGraph = document.querySelector(`#${graph} .js-plotly-plot`)
#             }
#             myGraph.on('plotly_animated', () => {
#                 localStorage.setItem(target, myGraph.layout.sliders[0].active)
#                 document.querySelector('#syncData').click()
#             })
#             return window.dash_clientside.no_update
#         }
#     """,
#     Output("currentFrame", "id"),
#     Input("myfig", "id"),
#     State("currentFrame", "id"),
# )

# app.clientside_callback(
#     """
#         function (n, store) {
#             return localStorage.getItem(store)
#         }
#     """,
#     Output('currentFrame', 'data'),
#     Input('syncData', 'n_clicks'),
#     State("currentFrame", "id"),
#     prevent_initial_call=True
# )

# @app.callback(
#     Output('toShow', 'children'),
#     Input('currentFrame', 'data'),
#     State("myfig", "figure"),
# )
# def toShow(d, fig):
#     return html.Div([
#         d,
#         '-- data --',
#         json.dumps(fig['frames'][int(d)])
#     ], style={'padding': '10px'})

# if __name__ == '__main__' :
#     app.run(debug=True)

In [ ]:
# import plotly.express as px

# df = sensors.left_df.copy()
# df['frame'] = np.arange(len(df))
# df['zeros'] = [0] * len(df)
# fig = px.scatter(df, x="elapsed_time", y="zeros", 
#                  animation_frame="elapsed_time", 
#                 )

# app = dash.Dash(__name__)
# # fig = go.Figure(
#     # data=[px.scatter(df, x='elapsed_time', y='zeros',
#     #                 #  name='Initial', #mode='lines', 
#     #                  animation_frame="elapsed_time",)],
#     # layout=go.Layout(
#     #     updatemenus=[dict(type="buttons",
#     #                       buttons=[dict(
#     #                         args = [None, {"frame": {"duration": 10, "redraw": False},
#     #                             "fromcurrent": True, "transition": {"duration": 10}}],
#     #                         label="Play",
#     #                         method="animate",
#     #                         )])]
#     # )
# # )
# app.layout = html.Div([
#     html.Div(children=[
#         dcc.Graph(id='Acc-Inputs', figure=fig, config={'displayModeBar': False}), 
#         html.Div(children=[
#                     html.Label('Select Sensors to Plot:'),
#                     dcc.Checklist(['Left Ankle', 'Right Ankle', 'Waist'],
#                                   ['Left Ankle', 'Right Ankle', 'Waist'],
#                                 id='selected-sensors', ), 
#                 ], style={'width': '30%', 'fontFamily': 'Arial', 'fontSize': 14, 
#                             'display': 'inline-block'}), 

#                 html.Div(children=[    
#                     html.Label('Window Size (seconds)'),
#                     dcc.Slider(
#                         min=1,
#                         max=15,
#                         marks={i: f'{i}' if i == 1 else str(i) for i in range(1, 16)},
#                         value=5,
#                         id='window-slider',
#                     ),
#                     ], style={'width': '30%', 
#                             'fontFamily': 'Arial', 'fontSize': 14, 
#                             'display': 'inline-block', 
#                             }
#         ), 
#     ]) 
# ]) 


# @app.callback(
#     Output('Acc-Inputs', 'figure'),
#     Input('selected-sensors', 'value'),
#     Input('window-slider', 'value'),
#     )

# def update_full_viz(selected_sensors: List[str], 
#                     window_size) -> go.Figure:
#     ''' Update the full visualization figure based on selected sensors and window size. '''

#     fig.update_layout(width=600, height=450,

#         updatemenus = [dict(type = "buttons",
#                 buttons = [
#                     dict(
#                         args = [None, {"frame": {"duration": 10, "redraw": False},
#                                         "fromcurrent": True, "transition": {"duration": 10}}],
#                         label = "Play",
#                         method = "animate",

#                         )])])


#     if 'Left Ankle' in selected_sensors:
#         fig.add_trace(go.Scatter(x=sensors.left_df['elapsed_time'], 
#                             y = sensors.left_df['accel_filtered'], 
#                             mode = 'lines', name='Left Ankle', 
#                             ))
        
#     if 'Right Ankle' in selected_sensors:
#         fig.add_trace(go.Scatter(x=sensors.right_df['elapsed_time'], 
#                             y = sensors.right_df['accel_filtered'], 
#                             mode = 'lines', name='Right Ankle', 
#                             ))


#     fig.update_layout(xaxis_range = [0-window_size, 0 + window_size])


#     # go.Figure(
#     #     data=fig.data,
#     #     frames=[
#     #         fr.update(
#     #             layout={
#     #                 "xaxis": {"range": [i-window_size, i + window_size]},
#     #             }
#     #         )
#     #         for i, fr in enumerate(fig.frames)
#     #     ],
#     #     layout=fig.layout,
#     # )
#     return fig

# app.run(debug=True, use_reloader=False, host='127.0.0.1', port=8050)

In [ ]:
#  def sensor_mock_realtime(sensors):
#     ''' Visualize sensor data as if collecting in real-time.

#     Args: 
#         sensors (SensorData): Sensor data object containing left and right leg data.

#     Returns:
#         None: Starts a Dash app to visualize the sensor data and vGRF estimations 
#         in real time.
            
#     '''

#     # setup plotly dash app and tab layout
#     app = dash.Dash(__name__)

#     app.layout = html.Div([
#         html.H1(children='Vetta Realtime vGRF Estimation',
#                  style={'textAlign': 'center', 
#                         'marginBottom': 20, 
#                         'fontFamily': 'Arial'}),

#         dcc.Tabs([
#             dcc.Tab(label='Acc Inputs',
#                     children=[
#                 dcc.Graph(id='Acc-Inputs'), 

#                 html.Div(children=[
#                     html.Label('Select Sensors to Plot:'),
#                     dcc.Checklist(['Left Ankle', 'Right Ankle', 'Waist'],
#                                   ['Left Ankle', 'Right Ankle', 'Waist'],
#                                 id='selected-sensors', ), 
#                 ], style={'width': '30%', 'fontFamily': 'Arial', 'fontSize': 14, 
#                             'display': 'inline-block'}), 

#                 html.Div(children=[    
#                     html.Label('Window Size (seconds)'),
#                     dcc.Slider(
#                         min=1,
#                         max=15,
#                         marks={i: f'{i}' if i == 1 else str(i) for i in range(1, 16)},
#                         value=5,
#                         id='window-slider',
#                     ),
#                     ], style={'width': '30%', 
#                             'fontFamily': 'Arial', 'fontSize': 14, 
#                             'display': 'inline-block', 
#                             }
#                 ),
             
#             ]),

#             dcc.Tab(label='Model Outputs', children=[
#                 dcc.Graph(id='Model Outputs', config={'displayModeBar': False}),
#                 # dcc.Interval(id='right-interval', interval=100, n_intervals=0)
#             ]),


#             dcc.Tab(label='Full Viz', children=[
#                 dcc.Graph(id='Full Viz', config={'displayModeBar': False}),
#                 # dcc.Interval(id='right-interval', interval=100, n_intervals=0)
#             ])
#         ]), 
#         html.Div(id='output-container', style={'marginTop': 10})
#     ])

 
#     @app.callback(
#         Output('Acc-Inputs', 'figure'),

#         Input('window-slider', 'value'),
#         Input('selected-sensors', 'value'),
#         Input('Acc-Inputs', 'relayoutData'),

#         )
    
#     def update_input_fig(window_size, selected_sensors, relayoutData):
#         ''' Update the input figure based on the selected window size. '''
#         # Update the figure with new data based on the window size
#         print('Window Size:', window_size, '  s \nSelected sensors:', selected_sensors)

#         # fig_dict = {

#         # fig = go.Figure(id='Acc-Inputs', config={'displayModeBar': False})
#         # # if selected_sensors:
#         # #     print(selected_sensors)

#         # if 'Left Ankle' in selected_sensors:
#         #     line_trace = go.Scatter(x=sensors.left_df['elapsed_time'], 
#         #                             y=sensors.left_df['accel_filtered'], 
#         #                             mode='lines', name='Left Ankle', 
#         #                             )
#         #     fig.add_trace(line_trace)

#         # if 'Right Ankle' in selected_sensors:
#         #     line_trace = go.Scatter(x=sensors.right_df['elapsed_time'], 
#         #                             y=sensors.right_df['accel_filtered'], 
#         #                             mode='lines', name='Right Ankle', 
#         #                             )
#         #     fig.add_trace(line_trace)

#         # if 'Waist' in selected_sensors:
#         #     line_trace = go.Scatter(x=sensors.waist_df['elapsed_time'], 
#         #                             y=sensors.waist_df['accel_filtered'], 
#         #                             mode='lines', name='Waist', 
#         #                             )
#         #     fig.add_trace(line_trace)


#         num_frames = len(sensors.left_df['elapsed_time'])
#         # frames = [go.Frame(data=[go.Scatter(x=sensors.left_df['elapsed_time'][:i+1],
#         #                                         y=sensors.left_df['accel_filtered'][:i+1])],
#         #                   name=str(i)) for i in range(num_frames)]

#         fps = 100

#         frame_duration = 1 / fps

#         start = 0
#         end_ind = np.where(sensors.left_df['elapsed_time'] >= start + window_size)[0][0]
#         end = sensors.left_df['elapsed_time'][end_ind]
#         print(end)

#         # fig.update_layout(
#         #     xaxis=dict(range=[start, end]),
#         #     title='Sensor Acceleration Inputs',
#         #     xaxis_title='Time (s)',
#         #     yaxis_title='Acceleration (m/s²)',
#         #     font=dict(family="Arial, sans-serif"), 
#         #     template="plotly_white",

#         #     updatemenus=[dict(
#         #             type = "buttons",
#         #             direction = "down",
#         #             buttons=list([

#         #                 {"args": [None, {"frame": {"duration": frame_duration, "redraw": False},
#         #                                 "fromcurrent": True, 
#         #                                 "transition": {"duration": 30,
#         #                                              "easing": "quadratic-in-out"}}],
#         #                 "label": "Play",
#         #                 "method": "animate"
#         #                 },
#         #                 {"args": [[None], {"frame": {"duration": 0, "redraw": False},
#         #                           "mode": "immediate",
#         #                           "transition": {"duration": 0}}],
#         #                 "label": "Pause",
#         #                 "method": "animate"
#         #                 }
#         #             #     dict(
#         #             #         args=["playback", "play"],
#         #             #         label="play",
#         #             #         method="animate"
#         #             #     ),
#         #             #     dict(
#         #             #         args=["playback", "pause"],
#         #             #         label="pause",
#         #             #         method="animate"
#         #             #     ),
#         #             #     dict(
#         #             #         args=["playback", "restart"],
#         #             #         label="restart",
#         #             #         method="animate", 
#         #             #     )
#         #             ]),
#         #             pad={"r": 10, "t": 5},
#         #             showactive=True,
#         #             x=1.03,
#         #             xanchor="left",
#         #             y=0.5,
#         #             yanchor="top",
#         #         ),
#         #     ], 
#         # )


#         # Create figure
#         fig = go.Figure(
#             data=[],
#             layout=go.Layout(
#                             # xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
#                             # yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
#                             title=dict(text="Moving Frenet Frame Along a Planar Curve"),
#                             hovermode="closest",
#                             updatemenus=[dict(type="buttons",
#                                             buttons=[
#                                                 dict(label="Play",
#                                                     method="animate",
#                                                     args=[None]),
#                                                 dict(label="Pause}",
#                                                     method="animate",
#                                                     args=[None])],
#                                 pad={"r": 10, "t": 10},
#                                 showactive=True,
#                                 x=1.03,
#                                 xanchor="left",
#                                 y=0.5,
#                                 yanchor="top",)]
#                             ),

#             frames=[go.Frame(
#                 data=[go.Scatter(
#                     x=sensors.left_df['elapsed_time'], 
#                     y=sensors.left_df['accel_filtered'], 
#                     mode='lines', name='Left Ankle', )
#                 ]) for k in range(num_frames)]

#         )
        

#         return fig
  

#     # def open_browser():
#     #     webbrowser.open_new("http://127.0.0.1:8050/")

#     # threading.Timer(1, open_browser).start()
#     # app.run(debug=True, use_reloader=False, host='0.0.0.0', port=8050)
#     app.run(debug=True, use_reloader=False, host='127.0.0.1', port=8050)


# sensor_mock_realtime(sensors)

In [ ]:

# # setup plotly dash app and tab layout
# app = dash.Dash(__name__)

# # Create Input Figure - Tab 1
# # contains acceleration data from left ankle, right ankle, and waist
# # window_size_time = 2 # seconds
# # start = 0
# # end = start + window_size_time
# # initial_x = np.arange(start, end)


# app.layout = html.Div([
#         html.H1(children='Vetta Realtime vGRF Estimation',
#                 style={'textAlign': 'center', 
#                     'marginBottom': 20, 
#                     'fontFamily': 'Arial'}),

#     dcc.Tabs([
#         dcc.Tab(label='Acc Inputs',
#                 children=[
#             dcc.Graph(id='Acc-Inputs'), 
#                     #   figure=initial_fig,
#                     #   config={'displayModeBar': False}),

#             # dcc.Interval(id='left-interval', interval=100, n_intervals=0
#             #              ),

#             html.Div(children=[
#                 html.Label('Checkboxes'),
#                 dcc.Checklist(['Left Ankle', 'Right Ankle', 'Waist'],
#                             ['Left Ankle', 'Right Ankle', 'Waist'], 
#                             id='selected-sensors',
#                 ),

#                 html.Br(),
#                 html.Label('Window Size (seconds)'),
#                 dcc.Slider(
#                     min=1,
#                     max=5,
#                     marks={i: f'Label {i}' if i == 1 else str(i) for i in range(1, 6)},
#                     # value=5,
#                     id='window-slider',
#                 ),
#                 # html.Br(),
#                 # html.Br(),
#                 ], style={'padding': 10, 'flex': 1})
            
#         ]),


#         dcc.Tab(label='Model Outputs', children=[
#             dcc.Graph(id='Model Outputs', config={'displayModeBar': False}),
#             # dcc.Interval(id='right-interval', interval=100, n_intervals=0)
#         ]),


#         dcc.Tab(label='Full Viz', children=[
#             dcc.Graph(id='Full Viz', config={'displayModeBar': False}),
#             # dcc.Interval(id='right-interval', interval=100, n_intervals=0)
#         ])
#     ]), 
#     html.Div(id='output-container', style={'marginTop': 20})
# ])


# @app.callback(
#     Output('Acc-Inputs', 'figure'),

#     Input('window-slider', 'value'),
#     Input('selected-sensors', 'value'),
#     #  Output('click-state', 'data')],
#     # [Input('window-slider', 'value'),
#     #  Input('time-series-plot', 'clickData'),
#     #  Input('time-series-plot', 'hoverData'),
#     #  Input('click-state', 'data'),
#     #  Input('offset-input', 'value')
#     #  ],
#     # prevent_initial_call=False,
#     )

# def update_input_fig(window_size, selected_sensors):
#     ''' Update the input figure based on the selected window size. '''
#     # Update the figure with new data based on the window size
#     start = 0
#     end = np.where(sensors.left_df['elapsed_time'] >= start + window_size)[0][0]
#     # end = start + window_size * 
#     # x_values = sensors.left_df['elapsed_time'][start:end]
#     # y_values = sensors.left_df['accel_filtered'][start:end]

#     # updated_fig = go.Figure()
#     # updated_fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='left ankle acc', line=dict(color='#6B4E71')))
#     fig = go.Figure()
#     if selected_sensors:
#         print(selected_sensors)
#     fig.add_trace(px.line(x = sensors.left_df['elapsed_time'][start:end],
#                             y = sensors.left_df['accel_filtered'][start:end],
#                     name='Left Ankle Acceleration',)
#     )

#     # initial_fig.update_layout(
#     #     xlimit=[start, end],
#     #     title='Sensor Acceleration Inputs',
#     #     xaxis_title='Time (s)',
#     #     yaxis_title='Acceleration (m/s²)',
#     #     font=dict(family="Arial, sans-serif")
#     # )

#     return fig


# # def open_browser():
# #     webbrowser.open_new("http://127.0.0.1:8050/")

# # threading.Timer(1, open_browser).start()
# # app.run(debug=True, use_reloader=False, host='0.0.0.0', port=8050)
# # app.run(debug=True, use_reloader=False, host='127.0.0.1', port=8050)


# if __name__ == '__main__':
#     # app.run(debug=True)
#     app.run(debug=True, use_reloader=False, host='127.0.0.1', port=8050)